In [2]:
import httpx
import os

In [13]:
with httpx.Client() as request_client:
    nominatim_response = request_client.get(
        "https://nominatim.openstreetmap.org/search.html?",
        params={
            "q": 'Auckland',
        },
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
        },
    )

if nominatim_response.status_code != 200:
    raise ValueError(
        f"Request failed with status code {nominatim_response.status_code}, response: {nominatim_response.headers, nominatim_response.text, nominatim_response.url, nominatim_response.request.url, nominatim_response.has_redirect_location, nominatim_response.is_error, nominatim_response.is_redirect, nominatim_response.reason_phrase, nominatim_response.status_code, nominatim_response.url}"
    )
nominatim_response = nominatim_response.json()
print(nominatim_response['results'])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [17]:
latitude = geocode_response['results'][0]['geometry']['location']['lat']
longitude = geocode_response['results'][0]['geometry']['location']['lng']

In [21]:
with httpx.Client() as request_client:
    tairua_geocode_response = request_client.get(
        "https://maps.googleapis.com/maps/api/geocode/json",
        params={
            "address": 'Tairua',
            "key": os.environ["GOOGLE_MAPS_API_KEY"]
        }
    )

if tairua_geocode_response.status_code != 200:
    raise ValueError(
        f"Request failed with status code {tairua_geocode_response.status_code}"
    )
tairua_geocode_response = tairua_geocode_response.json()


In [22]:
tairua_latitude = tairua_geocode_response['results'][0]['geometry']['location']['lat']
tairua_longitude = tairua_geocode_response['results'][0]['geometry']['location']['lng']

In [24]:
my_lat = -36.8511689
my_lon = 174.8530858

In [28]:
with httpx.Client() as request_client:
    geocode_response = request_client.get(
        "https://maps.googleapis.com/maps/api/geocode/json",
        params={
            "latlng": f"{latitude},{longitude}",
            "key": os.environ["GOOGLE_MAPS_API_KEY"],
            "result_type": "political"
        }
    )

if geocode_response.status_code != 200:
    raise ValueError(
        f"Request failed with status code {geocode_response.status_code}"
    )
geocode_response = geocode_response.json()

print(geocode_response['results'][0]['formatted_address'])

with httpx.Client() as request_client:
    tairua_geocode_response = request_client.get(
        "https://maps.googleapis.com/maps/api/geocode/json",
        params={
            "latlng": f"{tairua_latitude},{tairua_longitude
            }",
            "key": os.environ["GOOGLE_MAPS_API_KEY"],
            "result_type": "political"
        }
    )

if tairua_geocode_response.status_code != 200:
    raise ValueError(
        f"Request failed with status code {tairua_geocode_response.status_code}"
    )
tairua_geocode_response = tairua_geocode_response.json()
print(tairua_geocode_response['results'][0]['formatted_address'])

with httpx.Client() as request_client:
    my_geocode_response = request_client.get(
        "https://maps.googleapis.com/maps/api/geocode/json",
        params={
            "latlng": f"{my_lat},{my_lon}",
            "key": os.environ["GOOGLE_MAPS_API_KEY"],
            "result_type": "political"
        }
    )

if my_geocode_response.status_code != 200:
    raise ValueError(
        f"Request failed with status code {my_geocode_response.status_code}"
    )
my_geocode_response = my_geocode_response.json()
print(my_geocode_response['results'][0]['formatted_address'])

Auckland CBD, Auckland 1010, New Zealand
Tairua, New Zealand
St Heliers, Auckland, New Zealand
